In [1]:
import numpy as np
from scipy import optimize, special
import itertools

In [2]:
def initialize_weights(L_in, L_out):
    dt = np.dtype('f8')   # 64-bit floating-point number
    t = np.zeros((L_out, L_in + 1), dtype=dt)
    e = 0.12 # sigma
    t[:] = np.random.randn(*t.shape) * 2.0 * e - e
    
    print(f'generated theta: {np.shape(t)}')
#     t = np.random.random((L_out, L_in + 1)) * 2.0 * e - e
    return t


def recode_label(y,num_labels):
    m = np.shape(y)[0]
    out = np.zeros( ( num_labels, m ) ) #(num_labels, m)
    for i in range(0, m):
        out[int(y[i]-1), i] = 1
    return out


def param_unroll( nn_params, input_layer_size, hidden_layer_size, num_labels ):
    '''
    theta1 shape: (30, 785) (hidden_size, input_size + 1)
    theta2 shape: (26, 31)  (num_labels, hidden_size + 1)
    '''
    theta1_elems = ( input_layer_size + 1 ) * hidden_layer_size
    theta1_size  = ( input_layer_size + 1, hidden_layer_size  )
    theta1 = nn_params[:theta1_elems].T.reshape( theta1_size ).T

    

    theta2_size  = ( hidden_layer_size + 1, num_labels )
    theta2 = nn_params[theta1_elems:].T.reshape( theta2_size ).T

    return (theta1, theta2)

In [3]:
# Some math

def sigmoid(z):
    return special.expit(z)
    #return ( (1 / (1 + np.exp(-z))) )

def sigmoid_gradient(z):
    sig = sigmoid(z)
    return (sig * (1 - sig))
   #return (sigmoid(z) * (1 - sigmoid(z)))

print(sigmoid(0.0)) #should return 0.5
print(sigmoid_gradient(0.0)) # should return 0.25

0.5
0.25


In [4]:
def feed_forward(theta1, theta2, X, X_bias=None):
    '''
    a1 = (m, input_layer_size + 1), a2 = (m, hidden_layer_size + 1), a3= (m, num_labels)
    theta1 = (hidden_layer_size, input_layer_size + 1)
    theta2 = (num_labels, hidden_layer_size)
    '''
    one_rows = np.ones((1, np.shape(X)[0] ))
    a1 = np.r_[one_rows, X.T]  if X_bias is None else X_bias
    z2 = theta1.dot( a1 )
    a2 = sigmoid(z2)
    a2 = np.r_[one_rows, a2] 
    z3 = theta2.dot( a2 )
    a3 = sigmoid( z3 )
#     # Input layer
#     m, _ = np.shape(X)
#     one_rows = np.ones((1, np.shape(X)[0] ))
#     a1 = np.c_[np.ones((m,1)), X] # assigning a1 to X, and adding a bias (m, input_layer_size + 1)
#     # Hidden layer
#     z2 = a1.dot(theta1.T)
#     a2 = sigmoid(z2)
#     a2 = np.c_[np.ones((np.shape(a2)[0], 1)), a2] # bias for hidden layer
#     # Output layer
#     z3 = a2.dot(theta2.T)
#     a3 = sigmoid(z3) #a3 = h(x)
    
    return (a1, a2, a3, z2, z3)

In [5]:
def compute_cost( nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, _lambda, yk = None, X_bias = None ):
    
    theta1, theta2 = param_unroll( nn_params, input_layer_size, hidden_layer_size, num_labels )
    a1,a2,a3,z2,z3 = feed_forward(theta1, theta2, X, X_bias)
    
    if yk is None:
        yk = recode_label(y, num_labels)
        assert shape(yk) == shape(a3), "Error, shape of recoded y is different from a3"
    
    # J(theta) function: cross-entropy
    term1 = (-y_k * np.log(a3))
    term2 = (1 - y_k) * np.log(1 - a3)
    cost = np.sum(term1 + term2)/m
    # Regularization sum
    reg_term = np.sum(theta1 ** 2) + np.sum(theta2[:,1:] ** 2)
    reg_term = (_lambda/2/m) * reg_term
    return(cost + reg_term)

In [6]:
def compute_gradient( nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, _lambda, yk = None, X_bias = None ):
    m, n = np.shape(X)
    theta1, theta2 = param_unroll( nn_params, input_layer_size, hidden_layer_size, num_labels )
    a1, a2, a3, z2, z3 = feed_forward(theta1, theta2, X)
    
    # back propagate
    if yk is None:
        yk = recode_label(y, num_labels )
        assert shape(yk) == shape(a3), "Error: shape of recoded y is different from a3"
    
    #size of a3 == size of y_k == (num_labels, m)
    delta_3 = a3 - y_k 
    # check this line
    #delta_2 = (delta_3.dot(theta2))[:,1:] * sigmoid_gradient(z2) #ignore bias
    delta_2 = theta2.T.dot(delta_3) * sigmoid_gradient(np.r_[np.ones((1, m)), z2 ]) # WHAT
    delta_2 = delta_2[1:,:]
   
    sum_1 = delta_2.dot(a1.T)
    sum_2 = delta_3.dot(a2.T) # sum of a_i * delta_i+1
    
    # putting the gradient equation together
    sum_1[:,1:] = (sum_1[:,1:] / m) + (theta1[:,1:] * _lambda / m)
    sum_2[:,1:] = (sum_2[:,1:] / m) + (theta2[:,1:] * _lambda / m)
    
    accum = np.array([sum_1.T.reshape(-1).tolist() + sum_2.T.reshape(-1).tolist()]).T
    return np.ndarray.flatten(accum)


In [7]:
def numerical_gradient( theta, input_layer_size, hidden_layer_size, num_labels, X, y, lamda ):
    numgrad = np.zeros( np.shape(theta) )
    perturb = np.zeros( np.shape(theta) ) 
    e = 1e-4

    num_elements = np.shape(theta)[0]
    yk = recode_label( y, num_labels )

    for p in range(0, num_elements) :
        perturb[p] = e
        loss1 = compute_cost( theta - perturb, input_layer_size, hidden_layer_size, num_labels, X, y, lamda, yk )
        loss2 = compute_cost( theta + perturb, input_layer_size, hidden_layer_size, num_labels, X, y, lamda, yk )
        numgrad[p] = (loss2 - loss1) / (2 * e)
        perturb[p] = 0

    return numgrad

In [8]:
# loading input data
data = np.genfromtxt('./data/tt.csv', delimiter=',')
y = data[:,0].reshape(-1,1)
#X = data[:, 1:] / 255.0 #(m, input_layer_size)
X = data[:, 1:] #(m, input_layer_size)
m = len(y)

# Network architecture 
input_layer_size = 784
hidden_layer_size = 30
num_labels = 26
lam = 1.0

# Params
theta1 = initialize_weights( 784, 30 ) # (input_size, hidden layer 1 size)
theta2 = initialize_weights( 30, 26 )  # (hidden layer 1 size, # labels)
unrolled = np.r_[theta1.T.flatten(), theta2.T.flatten()] # 1 dimension (24356,)

#grad1 shape: (30, 784) 
#grad2 shape: (26, 30)


# matrix holding correct values
y_k = recode_label(y,num_labels)
print(f'shape of y_k: {np.shape(y_k)}')

X_bias = np.r_[ np.ones((1, np.shape(X)[0] )), X.T] #(input_size + 1, m)
print(np.shape(X_bias))


generated theta: (30, 785)
generated theta: (26, 31)
shape of y_k: (26, 688)
(785, 688)


In [9]:
# # Debugging the parameters before
# print(f'rolled params: {np.shape(unrolled)}')
# print(f'number of elements in theta1: {np.size(theta1)}')
# print(f'number of elements in theta2: {np.size(theta2)}')
# print(f'theta1 shape: {np.shape(theta1)}')
# print(f'theta2 shape: {np.shape(theta2)}')

# print('='*20)

# t1,t2 = param_unroll(unrolled, input_layer_size, hidden_layer_size, num_labels)

# # Debugging after:
# print(f'number of elements in theta1: {np.size(t1)}')
# print(f'number of elements in theta2: {np.size(t2)}')
# print(f'theta1 shape: {np.shape(t1)}')
# print(f'theta2 shape: {np.shape(t2)}')

# # debugging values are the same
# print(f'{theta1[0,0] == t1[0,0]},{theta1[12,25] == t1[12,25]},{theta1[11, 200] == t1[11,200]}')
# print(f'{theta2[0,0] == t2[0,0]},{theta2[12,25] == t2[12,25]},{theta2[11, 11] == t2[11,11]}')

In [10]:
# print(compute_cost( unrolled, input_layer_size, hidden_layer_size, num_labels, X, y, lam, y_k, X_bias))
# print('=' * 20)
# print(numerical_gradient( unrolled, input_layer_size, hidden_layer_size, num_labels, X, y, lam ))
# print('=' * 20)
print(compute_gradient( unrolled, input_layer_size, hidden_layer_size, num_labels, X, y, lam, y_k, X_bias))
# print('=' * 20)

[ 0.          0.          0.         ...  0.00037675 -0.0012299
 -0.00013283]


In [11]:
result = optimize.fmin_cg(compute_cost, fprime=compute_gradient, x0=unrolled,
    args=(input_layer_size, hidden_layer_size, num_labels, X, y, lam, y_k, X_bias),
    maxiter=400, disp=True, full_output=True )
# result = optimize.fmin_bfgs(compute_cost, fprime=compute_gradient, x0=unrolled,
#     args=(input_layer_size, hidden_layer_size, num_labels, X, y, lam, y_k, X_bias),
#     maxiter=50, disp=True, full_output=True )

# print(result)



In [12]:
# theta1, theta2 = param_unroll( result[0], input_layer_size, hidden_layer_size, num_labels )
# def predict(item, theta1, theta2):
#     a1 = np.r_[np.ones((1, 1)), X.reshape(np.shape(X)[0], 1 )]
#     z2 = sigmoid( theta1.dot( a1 ))
#     z2 = np.r_[np.ones((1, 1)), z2]
#     z3 = sigmoid(theta2.dot( z2 ))
#     return argmax(z3) + 1